In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
#buat dataframe untuk train dan test
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("test.csv")

In [ ]:
df_train

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,35.0,<=7jt
1,1,49,Wiraswasta,140782,10th,6,Cerai,Ekesekutif Managerial,Perempuan,0.0,0.0,40.0,<=7jt
2,2,44,Wiraswasta,120057,D3,12,Menikah,Ekesekutif Managerial,Laki2,61404000.0,0.0,45.0,>7jt
3,3,24,Wiraswasta,194630,Sarjana,13,Belum Pernah Menikah,Spesialis,Laki2,0.0,0.0,35.0,<=7jt
4,4,33,Wiraswasta,219619,Master,14,Menikah,Spesialis,Laki2,210336000.0,0.0,40.0,>7jt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35989,35989,47,Pekerja Bebas Bukan Perusahan,148169,SMA,9,Menikah,Perbaikan Kerajinan,Laki2,0.0,0.0,40.0,<=7jt
35990,35990,69,Pekerja Bebas Perusahaan,264722,D3,12,Menikah,Sales,Laki2,0.0,0.0,40.0,>7jt
35991,35991,24,Pekerja Bebas Bukan Perusahan,31606,Sarjana,13,Menikah,Spesialis,Perempuan,0.0,0.0,20.0,>7jt
35992,35992,47,Wiraswasta,197836,SMA,9,Menikah,Sales,Laki2,0.0,0.0,45.0,<=7jt


In [ ]:
df_val

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu
0,35994,30,Pemerintah Lokal,99761,D4,11,Cerai,Pemuka Agama,Perempuan,0.0,0.0,40.0
1,35995,28,Wiraswasta,202206,SMA,9,Menikah,Mesin Inspeksi,Laki2,0.0,0.0,50.0
2,35996,34,Pemerintah Provinsi,199934,Pendidikan Tinggi,10,Menikah,Pemuka Agama,Laki2,0.0,0.0,40.0
3,35997,51,Wiraswasta,373448,Pendidikan Tinggi,10,Menikah,Ekesekutif Managerial,Laki2,0.0,28028000.0,40.0
4,35998,20,?,144685,Pendidikan Tinggi,10,Belum Pernah Menikah,?,Perempuan,0.0,22428000.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9594,45588,58,Pemerintah Negara,159021,9th,5,Menikah,Petani,Laki2,0.0,0.0,40.0
9595,45589,45,Pekerja Bebas Bukan Perusahan,144086,11th,7,Menikah,Perbaikan Kerajinan,Laki2,0.0,0.0,50.0
9596,45590,20,Wiraswasta,238685,Pendidikan Tinggi,10,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,32.0
9597,45591,41,Pemerintah Lokal,249039,SMA,9,Berpisah,Pemuka Agama,Perempuan,0.0,0.0,40.0


In [ ]:
#cek data kosong
print("DATA TRAIN")
df_train.isnull().sum()

DATA TRAIN


id                       0
Umur                     0
Kelas Pekerja            0
Berat Akhir              0
Pendidikan               0
Jmlh Tahun Pendidikan    0
Status Perkawinan        0
Pekerjaan                0
Jenis Kelamin            0
Keuntungan Kapital       0
Kerugian Capital         0
Jam per Minggu           0
Gaji                     0
dtype: int64

In [ ]:
print("DATA VALIDATION")
df_val.isnull().sum()

DATA VALIDATION


id                       0
Umur                     0
Kelas Pekerja            0
Berat Akhir              0
Pendidikan               0
Jmlh Tahun Pendidikan    0
Status Perkawinan        0
Pekerjaan                0
Jenis Kelamin            0
Keuntungan Kapital       0
Kerugian Capital         0
Jam per Minggu           0
dtype: int64

In [ ]:
#cek missing value pada kolom prakerja
df_train['Kelas Pekerja'].value_counts()

Wiraswasta                       24948
Pekerja Bebas Bukan Perusahan     2866
Pemerintah Lokal                  2280
?                                 2063
Pemerintah Negara                 1477
Pekerja Bebas Perusahaan          1273
Pemerintah Provinsi               1066
Tanpa di Bayar                      15
Tidak Pernah Bekerja                 6
Name: Kelas Pekerja, dtype: int64

In [ ]:
df_val['Kelas Pekerja'].value_counts()

Wiraswasta                       6714
Pekerja Bebas Bukan Perusahan     732
Pemerintah Lokal                  624
?                                 552
Pemerintah Negara                 376
Pekerja Bebas Perusahaan          316
Pemerintah Provinsi               279
Tanpa di Bayar                      4
Tidak Pernah Bekerja                2
Name: Kelas Pekerja, dtype: int64

In [ ]:
df_train['Pekerjaan'].value_counts()

Spesialis                4587
Perbaikan Kerajinan      4507
Ekesekutif Managerial    4481
Pemuka Agama             4158
Sales                    4044
Servis Lainnya           3615
Mesin Inspeksi           2250
?                        2069
Supir                    1741
Pembersih                1540
Petani                   1076
Tech-support             1042
Penjaga                   686
Asisten Rumah Tangga      187
Tentara                    11
Name: Pekerjaan, dtype: int64

In [ ]:
df_val['Pekerjaan'].value_counts()

Perbaikan Kerajinan      1205
Ekesekutif Managerial    1188
Spesialis                1142
Pemuka Agama             1112
Sales                    1079
Servis Lainnya            982
Mesin Inspeksi            590
?                         554
Supir                     458
Pembersih                 398
Petani                    316
Tech-support              301
Penjaga                   228
Asisten Rumah Tangga       43
Tentara                     3
Name: Pekerjaan, dtype: int64

In [ ]:
#ubah value ? pada setiap nilai dan tambahkan ke modus 
#kolom kelas pekerja
df_train['Kelas Pekerja'] = df_train['Kelas Pekerja'].replace(['?'],"Wiraswasta")
df_val['Kelas Pekerja'] = df_val['Kelas Pekerja'].replace(['?'],"Wiraswasta")
#kolom pekerjaan
df_train['Pekerjaan'] = df_train['Pekerjaan'].replace(['?'],"Spesialis")
df_val['Pekerjaan'] = df_val['Pekerjaan'].replace(['?'],'Perbaikan Kerajinan')

In [ ]:
'''ubah nilai gaji menjadi data kategorikal dengan ketentuan:
   <= 7jt = 0,
   > 7 jt = 1
'''
df_train['Gaji'] = df_train['Gaji'].replace(['<=7jt','>7jt'],[0,1])

In [ ]:
#ubah data nilai pendidikan ke menjadi int
df_train['Pendidikan'].value_counts()
obj_dict={
    '1st-4th'              : 0,
    '5th-6th'              : 1,
    'SD'                   : 2,  
    '7th-8th'              : 3,   
    '9th'                  : 4,
    '10th'                 : 5,
    '11th'                 : 6, 
    '12th'                 : 7,
    'SMA'                  : 8,                 
    'Sekolah Professional' : 9,
    'D3'                   : 10,                      
    'D4'                   : 11,
    'Pendidikan Tinggi'    : 12,  
    'Sarjana'              : 13,
    'Master'               : 14,
    'Doktor'               : 15
}

In [ ]:
#one hot encoding 
df_dummy_train = pd.get_dummies(df_train, columns=['Kelas Pekerja','Status Perkawinan','Pekerjaan','Jenis Kelamin'])
df_dummy_train['Pendidikan'] = df_dummy_train['Pendidikan'].replace(obj_dict)
df_dummy_train

,id,Umur,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji,Kelas Pekerja_Pekerja Bebas Bukan Perusahan,Kelas Pekerja_Pekerja Bebas Perusahaan,Kelas Pekerja_Pemerintah Lokal,Kelas Pekerja_Pemerintah Negara,Kelas Pekerja_Pemerintah Provinsi,Kelas Pekerja_Tanpa di Bayar,Kelas Pekerja_Tidak Pernah Bekerja,Kelas Pekerja_Wiraswasta,Status Perkawinan_Belum Pernah Menikah,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Pekerjaan_Asisten Rumah Tangga,Pekerjaan_Ekesekutif Managerial,Pekerjaan_Mesin Inspeksi,Pekerjaan_Pembersih,Pekerjaan_Pemuka Agama,Pekerjaan_Penjaga,Pekerjaan_Perbaikan Kerajinan,Pekerjaan_Petani,Pekerjaan_Sales,Pekerjaan_Servis Lainnya,Pekerjaan_Spesialis,Pekerjaan_Supir,Pekerjaan_Tech-support,Pekerjaan_Tentara,Jenis Kelamin_Laki2,Jenis Kelamin_Perempuan
0,0,21,242912,8,9,0.0,0.0,35.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,1,49,140782,5,6,0.0,0.0,40.0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,44,120057,10,12,61404000.0,0.0,45.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,3,24,194630,13,13,0.0,0.0,35.0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,4,33,219619,14,14,210336000.0,0.0,40.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35989,35989,47,148169,8,9,0.0,0.0,40.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
35990,35990,69,264722,10,12,0.0,0.0,40.0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
35991,35991,24,31606,13,13,0.0,0.0,20.0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
35992,35992,47,197836,8,9,0.0,0.0,45.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
df_dummy_val = pd.get_dummies(df_val, columns=['Kelas Pekerja','Status Perkawinan','Pekerjaan', 'Jenis Kelamin'])
df_dummy_val['Pendidikan'] = df_dummy_val['Pendidikan'].replace(obj_dict)
df_dummy_val

,id,Umur,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Kelas Pekerja_Pekerja Bebas Bukan Perusahan,Kelas Pekerja_Pekerja Bebas Perusahaan,Kelas Pekerja_Pemerintah Lokal,Kelas Pekerja_Pemerintah Negara,Kelas Pekerja_Pemerintah Provinsi,Kelas Pekerja_Tanpa di Bayar,Kelas Pekerja_Tidak Pernah Bekerja,Kelas Pekerja_Wiraswasta,Status Perkawinan_Belum Pernah Menikah,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Pekerjaan_Asisten Rumah Tangga,Pekerjaan_Ekesekutif Managerial,Pekerjaan_Mesin Inspeksi,Pekerjaan_Pembersih,Pekerjaan_Pemuka Agama,Pekerjaan_Penjaga,Pekerjaan_Perbaikan Kerajinan,Pekerjaan_Petani,Pekerjaan_Sales,Pekerjaan_Servis Lainnya,Pekerjaan_Spesialis,Pekerjaan_Supir,Pekerjaan_Tech-support,Pekerjaan_Tentara,Jenis Kelamin_Laki2,Jenis Kelamin_Perempuan
0,35994,30,99761,11,11,0.0,0.0,40.0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,35995,28,202206,8,9,0.0,0.0,50.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,35996,34,199934,12,10,0.0,0.0,40.0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,35997,51,373448,12,10,0.0,28028000.0,40.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,35998,20,144685,12,10,0.0,22428000.0,40.0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9594,45588,58,159021,4,5,0.0,0.0,40.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
9595,45589,45,144086,6,7,0.0,0.0,50.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
9596,45590,20,238685,12,10,0.0,0.0,32.0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
9597,45591,41,249039,8,9,0.0,0.0,40.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
#split data feature dan target
X = df_dummy_train.drop(['id','Gaji'],axis=1)
y = df_dummy_train['Gaji']
X_val = df_dummy_val.drop('id',axis=1)

In [ ]:
#normalisasi
col_train         = X.columns
col_val           = X_val.columns
stdscalar         = StandardScaler()
data_scale_train  = stdscalar.fit_transform(X)
data_scale_val   = stdscalar.fit_transform(X_val)

data_train = pd.DataFrame(data_scale_train, columns=col_train)
data_val = pd.DataFrame(data_scale_val, columns=col_val)

In [ ]:
#training model
X_train, X_result, y_train, y_result = train_test_split(data_train, y, test_size=0.2, random_state=0)
model = DecisionTreeClassifier()
param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth': np.arange(1,30),
              'min_samples_leaf' : np.arange(1,5)
             }
rscv = RandomizedSearchCV(model, param_grid, scoring='roc_auc', cv=10)
rscv.fit(X_train,y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [ ]:
rscv.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'splitter': 'best'}

In [ ]:
rscv.best_score_

0.8781880750778435

In [ ]:
rscv.classes_

array([0, 1])

CONFUSION MATRIX MODEL

In [ ]:
y_predict = rscv.predict(X_result)
print(confusion_matrix(y_result,y_predict))
print(classification_report(y_result,y_predict))
print('nilai akurasinya adalah ',accuracy_score(y_result, y_predict))

[[5197  270]
 [ 855  877]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      5467
           1       0.76      0.51      0.61      1732

    accuracy                           0.84      7199
   macro avg       0.81      0.73      0.76      7199
weighted avg       0.84      0.84      0.83      7199

nilai akurasinya adalah  0.8437282955966107


IMPLEMENTASI TERHADAP DATA VALIDATION

In [ ]:
prediksi = rscv.predict_proba(X_val)
prediksi

array([[0.59259259, 0.40740741],
       [0.22137405, 0.77862595],
       [0.22137405, 0.77862595],
       ...,
       [0.59259259, 0.40740741],
       [0.59259259, 0.40740741],
       [0.22137405, 0.77862595]])